# Comparative LCA and Monte Carlo

This notebook does Monte Carlo comparative LCA calculations, for GWP only, writing the results to `results/samples_comparative_gwp_contributions.csv`.

Results are calculated for UKR, CM HTO, AM HTO (Ti jig) and AM HTO (steel jig).  In addition, "low carbon electricity" scenarios are defined which are relevant to the AM HTO cases.

Contribution analysis is integrated into the Monte Carlo runs: for each sample, additional calculations are performed to separate the overall impact score into contributions from different sub-chains of the process chain (see `Contribution analysis.ipynb` notebook).  This allows us to see more clearly what's changing between the Monte Carlo samples and the different scenarios.

In [1]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps

In [2]:
# Generate objects for analysis

bw.projects.set_current("default")

db_UKR = bw.Database('UKR')
db_AMHTO_jig_steel = bw.Database('AM HTO- jig steel')
db_AMHTO = bw.Database('AM HTO')
db_CMHTO = bw.Database('CM HTO')
cutoff38 = bw.Database('cutoff38')
electricity = bw.Database('Electricity')
anesthesia = bw.Database('Anesthesia')
material = bw.Database('Raw materials')

# This contain the whole life-cycle impact of the AM and CM HTO
UKR = db_UKR.get('663ac4ef3d314e739f29ec63ea2ca399')
CMHTO = db_CMHTO.get("2f3acaae6c4e4035b2d24c71725b17d8")
AMHTO = db_AMHTO.get("1a637b9baee74199b8027ffeb333279c") 
AMHTO_jig_steel = db_AMHTO_jig_steel.get("1a637b9baee74199b8027ffeb333279c") 
polishing = db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy1")
anesthesia_gases = anesthesia.get("a00fcd60e79b42bd9f4868e509375673")

all_demands = [{UKR: 1}, {CMHTO: 1}, {AMHTO: 1}, {AMHTO_jig_steel: 1}]

#all_demands = [ {CMHTO: 1}]
#all_demands = [{CMHTO: 1}]
lce_demands = [{AMHTO: 1}, {AMHTO_jig_steel: 1}]
d_label = ["UKR", "CM HTO", "AM HTO", "AM HTO (steel jig)"]


method = ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)')

In [3]:
method

('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100')

In [4]:
num_samples = 1000
max_level=10
cutoff =1e-6

In [5]:
material.get("5153107d093942aa988fb2497e76ea87_copy1")

'1- Ti6Al4V production' (kilogram, GLO, None)

In [6]:
CMHTO

'Implantation' (unit, GB, None)

## Presamples for electricity inputs

Goal is to replace the values for the inputs to {SLM, polishing} with electricity from {GB, CH}. 

There is no harm in overwriting the inputs to processes that are not actually being used in each scenario (e.g. no problem to also swap the inputs to the steel SLM for instruments, even when in the scenario that they are not used).

In [7]:
def samples_to_swap_inputs(processes_to_swap, swaps):
    data = []
    indices = []
    for process in processes_to_swap:
        for swap in swaps:
            new_exchange = swap["new_exchange"]
            new_value = swap["new_value"](process) 
            indices_to_include = (new_exchange, process, "technosphere")
            
            indices.append(indices_to_include)
            data.append(new_value)


    return np.array(data), indices


In [8]:
def get_elec_input(process, elec):
    for exchange in process.technosphere():
        if exchange.input == elec:
            return exchange 

    raise RuntimeError("not found")

In [9]:
elec_CH = electricity.get("f115d3e7dd5f261c41b6f41a7b5df4ff_copy1")
elec_GB = electricity.get("826d168b2214847a40d2707229194e67_copy1")
elec_CH_instruments = electricity.get("f115d3e7dd5f261c41b6f41a7b5df4ff_copy2")
elec_GB_instruments = electricity.get("826d168b2214847a40d2707229194e67_copy2")
a = db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy1")
a

'2- Polishing' (hour, GB, None)

In [10]:
data1, indices1 = samples_to_swap_inputs(
    processes_to_swap=[
        # Ti instruments: electricity to SLM and to polishing (fpr plate and insturments separetely)
        db_AMHTO.get("fa8649e486f344bdaf9e06ce1df2699a_copy5"), #SLM service- main parts   
        db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy1"), #polishing
        db_AMHTO.get("fa8649e486f344bdaf9e06ce1df2699a_copy7"), #SLM service warm up and cool down
        
    ],
    swaps=[
        {"new_exchange": elec_CH, "new_value": lambda process: get_elec_input(process, elec_GB).random_sample(n=num_samples)},
        {"new_exchange": elec_GB, "new_value": lambda process: np.zeros(num_samples)},
    ]
)


In [11]:
data2, indices2 = samples_to_swap_inputs(
     processes_to_swap=[
        # Ti instruments: electricity to SLM and to polishing (fpr plate and insturments separetely)
        db_AMHTO.get("fa8649e486f344bdaf9e06ce1df2699a"), #SLM service- instruments
        db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy2"), #polishing
        db_AMHTO_jig_steel.get("fa8649e486f344bdaf9e06ce1df2699a"),   #SLM service- steel instruments
        db_AMHTO.get("fa8649e486f344bdaf9e06ce1df2699a_copy8"), #SLM service warm up and cool down
    ],
    
    swaps=[
        {"new_exchange": elec_CH_instruments, "new_value": lambda process: get_elec_input(process, elec_GB_instruments).random_sample(n=num_samples)},
        {"new_exchange": elec_GB_instruments, "new_value": lambda process: np.zeros(num_samples)},
    ]
)
data_combined = np.vstack((data1, data2))
indices_combined = indices1 + indices2 

low_carbon_electricity_matrix_data = [
    (data_combined, indices_combined, 'technosphere')]

low_carbon_electricity_matrix_data

[(array([[19.40680294, 15.44076139, 22.4194094 , ..., 23.49482524,
          19.25291418, 18.10874976],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 2.149938  ,  1.57180061,  1.72356704, ...,  1.72603064,
           1.2513517 ,  1.13211372],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 2.0829049 ,  2.07473153,  2.04740799, ...,  2.09161225,
           2.06062636,  2.00128746],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]]),
  [('market for electricity, medium voltage, machining' (kilowatt hour, CH, None),
    '2- SLM service per material added ' (kilogram, GB, None),
    'technosphere'),
   ('market for electricity, medium voltage, machining' (kilowatt hour, GB, None),
    '2- SLM service per material added ' (kilogram, GB, None),
    'technosphere'),
   ('market for electricit

Now define the values for these. The first of each pair will be set to the random samples, the second (GB) will be set to zero -- this is the opposite of how the processes are initially defined:

It's the same for SLM and polishing:

In [12]:
# Create presamples

low_carbon_electricity_id, low_carbon_electricity_path = ps.create_presamples_package(
    matrix_data = low_carbon_electricity_matrix_data,
)

## LCA calculation

In [13]:
from bw_helpers import sample_comparative_contribution, MyMonteCarloLCA, collect_contribution_samples

In [14]:
all_demand_dict = all_demands[0].copy()
for other_demand in all_demands[1:]:
    all_demand_dict.update(other_demand)
    
clca = MyMonteCarloLCA(all_demand_dict, method=method)

# Only modelling low-carbon electricity for the two AM devices at the moment
lce_demand_dict = lce_demands[0].copy()
for other_demand in lce_demands[1:]:
    lce_demand_dict.update(other_demand)
    
clca_low_carbon_electricity = MyMonteCarloLCA(
    lce_demand_dict,
    method=method,
    presamples=[low_carbon_electricity_path]
)

In [15]:
%%time
# Get things initialised
next(clca)
next(clca_low_carbon_electricity)

CPU times: total: 6.05 s
Wall time: 1.76 s


15.604965540615328

In [16]:
# These are the specific activities to break down the contributions to.
# In a separate file so can be shared between notebooks.
# import importlib
# importlib.reload(final_activities)

In [17]:
from final_activities import final_activities, component_order

In [18]:
%time sample_comparative_contribution(clca, all_demands, final_activities, cutoff= cutoff, max_level=max_level)

CPU times: total: 4.78 s
Wall time: 631 ms


[{'Total': 36.16458156546386,
  'Implant (material)': 23.166708360859207,
  'Implant (manufac.)': 4.897498757030254,
  'Argon': 0.001053948605032561,
  'Instruments (material)': 0.3106576143401987,
  'Instruments (manufac.)': 0.005005164087896073,
  'Packaging': 0.20945626602954945,
  'Sterilisation': 0.38736472194888366,
  'Transport': 3.7063140693855354,
  'Anesthesia': 3.4805226631772768},
 {'Total': 10.20364531406662,
  'Implant (material)': 4.87627449897081,
  'Implant (manufac.)': 0.458675349162381,
  'Argon': 2.6348713208692783e-05,
  'Instruments (material)': 0.3106576143401987,
  'Instruments (manufac.)': 0.005005164087896073,
  'Packaging': 0.21178721077021218,
  'Sterilisation': 0.20442499712935228,
  'Transport': 0.6562714677152743,
  'Anesthesia': 3.4805226631772768},
 {'Total': 11.230165495270963,
  'Implant (material)': 1.9654511967822987,
  'Implant (manufac.)': 1.0701942178569637,
  'Argon': 1.2078825192557596,
  'Instruments (material)': 3.108543237541582,
  'Instrume

At the start, wall time was 42s (total 1m5s). Adding factorisation was a dead end (we're using the iterative solver for MC), but improving caching of previous guesses when the demand is changing seems to have brought it down to about 4s (total 5.5s).

It seems to be mostly able to solve iteratively, occasionally has to restart from scratch.

At that rate, 100 samples will take 7 minutes.

In [19]:
%%time
samples = collect_contribution_samples(
    clca, 
    all_demands, 
    final_activities, 
    num_samples=num_samples, 
    method_label=method[1],
    demand_labels=d_label,
    component_order=component_order,
    max_level=max_level,
    cutoff= cutoff,
)


CPU times: total: 4h 51min 46s
Wall time: 2h 13min 30s


In [20]:
from bw_helpers import ScoreGrouper
grouper = ScoreGrouper(clca)
residual = grouper.residual_score() 
#print(residual)
#for key,score in grouper.residual_activities(exclude_databases={"cutoff38"}):
 #       print(f" {score}{score/residual:5.0%}  {bd.get_activity(key)}")

In [21]:
%%time
samples_lce = collect_contribution_samples(
    clca_low_carbon_electricity, 
    lce_demands, 
    final_activities, 
    num_samples=num_samples, 
    method_label=method[1],
    demand_labels=['AM HTO', 'AM HTO (steel jig)'],
    component_order=component_order,
    max_level=max_level,
    cutoff= cutoff
)

CPU times: total: 2h 37min 55s
Wall time: 1h 8min 33s


In [22]:
samples["energy_scenario"] = "Current"
samples_lce["energy_scenario"] = "Greener"
samples_combined = pd.concat([samples, samples_lce])

In [23]:
!mkdir -p results
samples_combined.to_csv("results/samples_comparative_gwp_contributions.csv", index=False)

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file results already exists.
Error occurred while processing: results.
